In [1]:
import os
os.chdir("/home/elicer/ptta") # os.chdir("/home/ubuntu/test-time-adapters")

In [2]:
# Set CUDA Device Number
DEVICE_NUM = 0

from os import environ
environ["CUDA_VISIBLE_DEVICES"] = str(DEVICE_NUM)

In [3]:
import sys
from os import path, environ
from argparse import ArgumentParser

import torch

from ttadapters import datasets, models
from ttadapters.models.base import ModelProvider
from ttadapters.utils import visualizer, validator
from ttadapters.datasets import DatasetHolder, scenarios

from ttadapters.methods.other_method.baseline import ActMADConfig, ActMAD, NORMConfig, NORM, DUAConfig, DUA, MeanTeacherConfig, MeanTeacher, WHWConfig, WHW

/home/elicer/ptta/.venv/lib/python3.10/site-packages/detectron2/model_zoo/model_zoo.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [4]:
environ["TORCHDYNAMO_CAPTURE_SCALAR_OUTPUTS"] = "1"
environ["TORCHDYNAMO_CAPTURE_DYNAMIC_OUTPUT_SHAPE_OPS"] = "1"

torch._dynamo.config.capture_scalar_outputs = True
torch._dynamo.config.suppress_errors = True

In [5]:
# Set Batch Size
BATCH_SIZE = 2, 8, 4  # Local
#BATCH_SIZE = 40, 200, 1  # A100 or H100
ACCUMULATE_STEPS = 1

# Set Data Root
DATA_ROOT = path.join(".", "data")

# Set Target Dataset
SOURCE_DOMAIN = datasets.SHIFTDataset

# Set Run Mode
TEST_MODE = True

# Set Model List
MODEL_ZOO = ["rcnn", "swinrcnn", "yolo11", "rtdetr"]
MODEL_TYPE = MODEL_ZOO[0]

In [6]:
# Set CUDA Device Number
DEVICE_NUM = 0
ADDITIONAL_GPU = 0
DATA_TYPE = torch.float32

if torch.cuda.is_available():
    if ADDITIONAL_GPU:
        torch.cuda.set_device(DEVICE_NUM)
        device = torch.device("cuda")
    else:
        device = torch.device(f"cuda:{DEVICE_NUM}")
else:
    device = torch.device("cpu")
    DEVICE_NUM = -1

print(f"INFO: Using device - {device}" + (f":{DEVICE_NUM}" if ADDITIONAL_GPU else ""))
print(f"INFO: Using data precision - {DATA_TYPE}")

INFO: Using device - cuda:0
INFO: Using data precision - torch.float32


In [7]:
# Fast download patch
datasets.patch_fast_download_for_object_detection()

In [8]:
# Basic pre-training dataset
match SOURCE_DOMAIN:
    case datasets.SHIFTDataset:
        dataset = DatasetHolder(
            train=datasets.SHIFTClearDatasetForObjectDetection(root=DATA_ROOT, train=True),
            valid=datasets.SHIFTClearDatasetForObjectDetection(root=DATA_ROOT, valid=True),
            test=datasets.SHIFTCorruptedDatasetForObjectDetection(root=DATA_ROOT, valid=True)
        )
    case datasets.CityscapesDataset:
        pass
    case _:
        raise ValueError(f"Unsupported dataset: {SOURCE_DOMAIN}")

# Dataset info
CLASSES = dataset.train.classes
NUM_CLASSES = len(CLASSES)
print(f"INFO: Number of classes - {NUM_CLASSES} {CLASSES}")

[10/13/2025 02:58:04] SHIFT DevKit - INFO - Base: ./data/SHIFT/discrete/images/train. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7f2de2fe0220>
[10/13/2025 02:58:04] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/normal/discrete/images/train/front/det_2d.json' ...


INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.


[10/13/2025 02:58:06] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/normal/discrete/images/train/front/det_2d.json' Done.
[10/13/2025 02:58:10] SHIFT DevKit - INFO - Loading annotation takes 6.36 seconds.


Batch 0:

Item                 Shape                               Min        Max       
--------------------------------------------------------------------------------
original_hw          [tensor([800]), tensor([1280])]
input_hw             [tensor([800]), tensor([1280])]
frame_ids            torch.Size([1])                           0.00       0.00
name                 ['00000000_img_front.jpg']
videoName            ['0016-1b62']
intrinsics           torch.Size([1, 3, 3])                     0.00     640.00
extrinsics           torch.Size([1, 4, 4])                    -7.53     219.91
boxes2d              torch.Size([1, 26, 4])                    5.00     974.00
boxes2d_classes      torch.Size([1, 26])                       0.00       3.00
boxes2d_track_ids    torch.Size([1, 26])                       0.00      25.00
images               torch.Size([1, 3, 800, 1280])             0.00     255.00



[10/13/2025 02:58:13] SHIFT DevKit - INFO - Base: ./data/SHIFT/discrete/images/val. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7f2de2fe0220>
[10/13/2025 02:58:13] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/normal/discrete/images/val/front/det_2d.json' ...


Video name: 0016-1b62
Sample indices within a video: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.


[10/13/2025 02:58:13] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/normal/discrete/images/val/front/det_2d.json' Done.
[10/13/2025 02:58:14] SHIFT DevKit - INFO - Loading annotation takes 0.63 seconds.
[10/13/2025 02:58:14] SHIFT DevKit - INFO - Base: ./data/SHIFT/discrete/images/val. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7f2de2fe0220>
[10/13/2025 02:58:14] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/corrupted/discrete/images/val/front/det_2d.json' ...


INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.


[10/13/2025 02:58:15] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/corrupted/discrete/images/val/front/det_2d.json' Done.
[10/13/2025 02:58:20] SHIFT DevKit - INFO - Loading annotation takes 5.96 seconds.


INFO: Dataset loaded successfully. Number of samples - Train: 20800, Valid: 2800, Test: 22200

INFO: Number of classes - 6 ['pedestrian', 'car', 'truck', 'bus', 'motorcycle', 'bicycle']


In [9]:
# Initialize model
match MODEL_TYPE:
    case "rcnn":
        model = models.FasterRCNNForObjectDetection(dataset=SOURCE_DOMAIN)
        load_result = model.load_from(**vars(model.Weights.SHIFT_CLEAR_NATUREYOO if TEST_MODE else model.Weights.IMAGENET_OFFICIAL), strict=False)
    case "swinrcnn":
        model = models.SwinRCNNForObjectDetection(dataset=SOURCE_DOMAIN)
        load_result = model.load_from(**vars(model.Weights.SHIFT_CLEAR_NATUREYOO if TEST_MODE else model.Weights.IMAGENET_XIAOHU2015), strict=False)
    case "rtdetr":
        model = models.RTDetrForObjectDetection(dataset=SOURCE_DOMAIN)
        load_result = model.load_from(**vars(model.Weights.SHIFT_CLEAR if TEST_MODE else model.Weights.COCO_OFFICIAL), strict=False)
    case _:
        raise ValueError(f"Unsupported model type: {MODEL_TYPE}")

print("INFO: Model state loaded -", load_result)
model.to(device)

INFO: Model state loaded - <All keys matched successfully>


FasterRCNNForObjectDetection(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2): Sequential(
        (0): Bo

In [10]:
# config = ActMADConfig()
# config.data_root = DATA_ROOT
# config.lr = 0.0001
# config.clear_dataset = model.DataPreparation(dataset.train, evaluation_mode=True)
# adaptive_model = ActMAD(model, config)

In [11]:
# config = NORMConfig()
# config.data_root = DATA_ROOT
# adaptive_model = NORM(model, config)

In [12]:
# config = DUAConfig()
# config.data_root = DATA_ROOT
# adaptive_model = DUA(model, config)

In [13]:
# config = MeanTeacherConfig()
# config.data_root = DATA_ROOT
# config.lr = 0.0001
# adaptive_model = MeanTeacher(model, config)

In [14]:
config = WHWConfig()
config.data_root = DATA_ROOT
config.lr = 0.0001
config.clear_dataset = model.DataPreparation(dataset.train, evaluation_mode=True)
adaptive_model = WHW(model, config)

WHW: Adapting 130 parameters with strategy 'adapter'
WHW: Total trainable parameters: 731,900


In [15]:
# # Compile model
# adaptive_model = torch.compile(adaptive_model)

In [16]:
# Ensure split (required due to Scenario class works with coroutines)
_ = datasets.SHIFTContinuousSubsetForObjectDetection(root=DATA_ROOT, train=True)

[10/13/2025 02:58:25] SHIFT DevKit - INFO - Base: ./data/SHIFT/continuous/images/1x/train. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7f2de2fe0220>
[10/13/2025 02:58:25] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/daytime_to_night/continuous/images/1x/train/front/det_2d.json' ...


INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/continuous/1x...
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/continuous/1x...
INFO: Dataset archive found in the root directory. Skipping download.


[10/13/2025 02:58:26] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/daytime_to_night/continuous/images/1x/train/front/det_2d.json' Done.
[10/13/2025 02:58:26] SHIFT DevKit - INFO - Loading annotation takes 0.84 seconds.


In [17]:
data_preparation = model.DataPreparation(datasets.base.BaseDataset(), evaluation_mode=True)

discrete_scenario = scenarios.SHIFTDiscreteScenario(
    root=DATA_ROOT, valid=True, order=scenarios.SHIFTDiscreteScenario.WHWPAPER, transforms=data_preparation.transforms
)
continuous_scenario = scenarios.SHIFTContinuousScenario(
    root=DATA_ROOT, valid=True, order=scenarios.SHIFTContinuousScenario.DEFAULT, transforms=data_preparation.transforms
)

[10/13/2025 02:58:26] SHIFT DevKit - INFO - Base: ./data/SHIFT/discrete/images/val. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7f2de2fe0220>
[10/13/2025 02:58:26] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/cloudy_daytime/discrete/images/val/front/det_2d.json' ...


INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.


[10/13/2025 02:58:27] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/cloudy_daytime/discrete/images/val/front/det_2d.json' Done.
[10/13/2025 02:58:27] SHIFT DevKit - INFO - Loading annotation takes 0.84 seconds.
[10/13/2025 02:58:27] SHIFT DevKit - INFO - Base: ./data/SHIFT/discrete/images/val. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7f2de2fe0220>
[10/13/2025 02:58:27] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/overcast_daytime/discrete/images/val/front/det_2d.json' ...


INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.


[10/13/2025 02:58:27] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/overcast_daytime/discrete/images/val/front/det_2d.json' Done.
[10/13/2025 02:58:28] SHIFT DevKit - INFO - Loading annotation takes 0.73 seconds.
[10/13/2025 02:58:28] SHIFT DevKit - INFO - Base: ./data/SHIFT/discrete/images/val. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7f2de2fe0220>
[10/13/2025 02:58:28] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/foggy_daytime/discrete/images/val/front/det_2d.json' ...


...

ContinuousSubsetType.DAYTIME_TO_NIGHT
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/continuous/1x...
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/continuous/1x...
INFO: Dataset archive found in the root directory. Skipping download.


[10/13/2025 02:58:32] SHIFT DevKit - INFO - Loading annotation takes 0.26 seconds.
[10/13/2025 02:58:32] SHIFT DevKit - INFO - Base: ./data/SHIFT/continuous/images/10x/val. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7f2de2fe0220>
[10/13/2025 02:58:32] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/daytime_to_night/continuous/images/10x/val/front/det_2d.json' ...
[10/13/2025 02:58:32] SHIFT DevKit - INFO - Base: ./data/SHIFT/continuous/images/100x/val. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7f2de2fe0220>
[10/13/2025 02:58:32] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/daytime_to_night/continuous/images/100x/val/front/det_2d.json' ...
[10/13/2025 02:58:32] SHIFT DevKit - INFO - Base: ./data/SHIFT/continuous/images/1x/val. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7f2de2fe0220>
[10/13/2025 02:58:32] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/clear_to_foggy/continuous/images/1

ContinuousSubsetType.DAYTIME_TO_NIGHT
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/continuous/10x...
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/continuous/10x...
INFO: Dataset archive found in the root directory. Skipping download.
ContinuousSubsetType.DAYTIME_TO_NIGHT
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/continuous/100x...
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/continuous/100x...
INFO: Dataset archive found in the root directory. Skipping download.
ContinuousSubsetType.CLEAR_TO_FOGGY
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/continuous/1x...
INFO: Dataset archive found in the ro

[10/13/2025 02:58:33] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/clear_to_rainy/continuous/images/1x/val/front/det_2d.json' Done.
[10/13/2025 02:58:33] SHIFT DevKit - INFO - Loading annotation takes 0.45 seconds.
[10/13/2025 02:58:33] SHIFT DevKit - INFO - Base: ./data/SHIFT/continuous/images/10x/val. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7f2de2fe0220>
[10/13/2025 02:58:33] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/clear_to_rainy/continuous/images/10x/val/front/det_2d.json' ...
[10/13/2025 02:58:33] SHIFT DevKit - INFO - Base: ./data/SHIFT/continuous/images/100x/val. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7f2de2fe0220>
[10/13/2025 02:58:33] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/clear_to_rainy/continuous/images/100x/val/front/det_2d.json' ...


ContinuousSubsetType.CLEAR_TO_RAINY
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/continuous/10x...
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/continuous/10x...
INFO: Dataset archive found in the root directory. Skipping download.
ContinuousSubsetType.CLEAR_TO_RAINY
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/continuous/100x...
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/continuous/100x...
INFO: Dataset archive found in the root directory. Skipping download.


In [18]:
evaluator = validator.DetectionEvaluator(adaptive_model, no_grad=False, classes=CLASSES, data_preparation=data_preparation, dtype=DATA_TYPE, device=device)
evaluator_loader_params = dict(batch_size=BATCH_SIZE[2], shuffle=False, collate_fn=data_preparation.collate_fn)

In [19]:
adaptive_model.model_provider = model.model_provider

In [20]:
visualizer.visualize_metrics(discrete_scenario(**evaluator_loader_params).play(evaluator, index=[adaptive_model.__class__.__name__]))

Output()

SHIFT Discrete Scenario:   0%|          | 0/7 [00:00<?, ?it/s]

Evaluation for cloudy_daytime:   0%|          | 0/600 [00:00<?, ?it/s]

AssertionError: get_event_storage() has to be called inside a 'with EventStorage(...)' context!

In [ ]:
visualizer.visualize_metrics(continuous_scenario(**evaluator_loader_params).play(evaluator, index=[adaptive_model.__class__.__name__]))